#Project Title: Holt–Winters Finetuned Ensemble for Adaptive Forecasting

#Tagline:
Finetuned Holt–Winters ensemble delivering adaptive, early‑accurate forecasts with robust diagnostic insights.

#Project Purpose: To develop and evaluate finetuned Holt–Winters ensemble trackers for adaptive, robust time series forecasting with improved early prediction accuracy and diagnostic clarity.

In [1]:
import warnings
warnings.filterwarnings('ignore')

#Install

In [2]:
!pip install birdgame river --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 108.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.2/354.2 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.1 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.


#Modelling and Deployment Simulation Section

In [ ]:
# ============================
# Holt & Holt-Winters Trackers (Mixture Output, Aggressive Tuning)
# CrunchDAO-ready single block (corrected)
# ============================

import os, math, numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import deque
from tqdm.auto import tqdm

# Birdgame imports
from birdgame.trackers.trackerbase import TrackerBase
from birdgame import HORIZON
from birdgame.visualization.utils import get_loc_and_scale
from birdgame.datasources.remotetestdata import remote_test_data_generator
from birdgame.trackers.tracker_evaluator import TrackerEvaluator

# ----------------------------
# EWMA utility
# ----------------------------
class EWMA:
    def __init__(self, alpha=0.1, init=0.0):
        self.alpha = alpha
        self.value = init
        self.initialized = False

    def update(self, x):
        if not self.initialized:
            self.value = x
            self.initialized = True
        else:
            self.value = (1 - self.alpha) * self.value + self.alpha * x

    def get(self, default=0.0):
        return self.value if self.initialized else default

# ----------------------------
# Constants
# ----------------------------
class Constants:
    VARIANCE_FLOOR = 0.05
    WARMUP_CUTOFF = 20
    MAX_SCALE_MULT = 4.0
    MIN_SCALE = 0.02
    RESIDUAL_EWMA_ALPHA = 0.2   # more aggressive

# ----------------------------
# Holt’s Linear Trend Tracker
# ----------------------------
class HoltTracker(TrackerBase):
    def __init__(self, horizon=HORIZON, alpha=0.7, beta=0.25, scale_alpha=Constants.RESIDUAL_EWMA_ALPHA):
        super().__init__(horizon)
        self.alpha, self.beta = float(alpha), float(beta)
        self.l, self.b, self.t = None, None, 0
        self.res_ewma = EWMA(alpha=scale_alpha)
        self.warmup = Constants.WARMUP_CUTOFF

    def initialize(self, x):
        self.l, self.b = float(x), 0.0

    def update_params(self, x):
        if self.l is None:
            self.initialize(x)
            return
        prev_l = self.l
        self.l = self.alpha * x + (1 - self.alpha) * (self.l + self.b)
        self.b = self.beta * (self.l - prev_l) + (1 - self.beta) * self.b

    def tick(self, payload, performance_metrics=None):
        x = float(payload['dove_location'])
        t = payload['time']
        self.add_to_quarantine(t, x)
        prev_x = self.pop_from_quarantine(t)

        self.update_params(x)
        if self.l is not None and self.b is not None and prev_x is not None:
            resid = x - self.l
            self.res_ewma.update(abs(resid))
        self.t += 1

    def predict(self):
        if self.t < self.warmup or self.l is None or self.b is None:
            return None
        loc = self.l + self.b
        base_scale = max(self.res_ewma.get(default=Constants.MIN_SCALE), Constants.MIN_SCALE)
        scale = min(max(base_scale, Constants.VARIANCE_FLOOR), Constants.MAX_SCALE_MULT * base_scale)
        return {
            "type": "mixture",
            "components": [
                {"density": {"type":"builtin","name":"norm","params":{"loc":float(loc),"scale":float(scale)}}, "weight":0.4},
                {"density": {"type":"builtin","name":"norm","params":{"loc":float(loc),"scale":float(scale*2)}}, "weight":0.6}
            ]
        }

# ----------------------------
# Holt–Winters Additive Tracker
# ----------------------------
class HoltWintersAddTracker(TrackerBase):
    def __init__(self, horizon=HORIZON, alpha=0.6, beta=0.2, gamma=0.3, m=12, scale_alpha=Constants.RESIDUAL_EWMA_ALPHA):
        super().__init__(horizon)
        self.alpha, self.beta, self.gamma, self.m = float(alpha), float(beta), float(gamma), int(m)
        self.l, self.b, self.s, self.t = None, None, deque(maxlen=self.m), 0
        self.res_ewma = EWMA(alpha=scale_alpha)
        self.warmup = max(Constants.WARMUP_CUTOFF, self.m + 5)

    def initialize(self, x):
        self.l, self.b = float(x), 0.0
        while len(self.s) < self.m:
            self.s.append(0.0)

    def seasonal_at_h(self, h):
        idx = (self.t - self.m + h) % self.m
        return list(self.s)[idx]

    def tick(self, payload, performance_metrics=None):
        x = float(payload['dove_location'])
        t = payload['time']
        self.add_to_quarantine(t, x)
        prev_x = self.pop_from_quarantine(t)

        if self.l is None or self.b is None or len(self.s) < self.m:
            self.initialize(x)

        s_tm = self.s[0] if len(self.s) == self.m else 0.0
        prev_l = self.l
        self.l = self.alpha * (x - s_tm) + (1 - self.alpha) * (self.l + self.b)
        self.b = self.beta * (self.l - prev_l) + (1 - self.beta) * self.b
        s_t = self.gamma * (x - self.l) + (1 - self.gamma) * s_tm
        if len(self.s) == self.m:
            self.s.popleft()
        self.s.append(s_t)

        if prev_x is not None:
            resid = x - (self.l + self.b + self.seasonal_at_h(1))
            self.res_ewma.update(abs(resid))
        self.t += 1

    def predict(self):
        if self.t < self.warmup or self.l is None or self.b is None or len(self.s) < self.m:
            return None
        loc = self.l + self.b + self.seasonal_at_h(1)
        base_scale = max(self.res_ewma.get(default=Constants.MIN_SCALE), Constants.MIN_SCALE)
        scale = min(max(base_scale, Constants.VARIANCE_FLOOR), Constants.MAX_SCALE_MULT * base_scale)
        return {
            "type":"mixture",
            "components":[
                {"density":{"type":"builtin","name":"norm","params":{"loc":float(loc),"scale":float(scale)}}, "weight":0.4},
                {"density":{"type":"builtin","name":"norm","params":{"loc":float(loc),"scale":float(scale*2)}}, "weight":0.6}
            ]
        }

# ----------------------------
# Holt–Winters Multiplicative Tracker
# ----------------------------
class HoltWintersMulTracker(TrackerBase):
    def __init__(self, horizon=HORIZON, alpha=0.7, beta=0.25, gamma=0.3, m=8, scale_alpha=Constants.RESIDUAL_EWMA_ALPHA):
        super().__init__(horizon)
        self.alpha, self.beta, self.gamma, self.m = float(alpha), float(beta), float(gamma), int(m)
        self.l, self.b, self.s, self.t = None, None, deque(maxlen=self.m), 0
        self.res_ewma = EWMA(alpha=scale_alpha)
        self.warmup = max(Constants.WARMUP_CUTOFF, self.m + 5)

    def initialize(self, x):
        self.l, self.b = max(float(x), 1e-6), 0.0
        while len(self.s) < self.m:
            self.s.append(1.0)

    def seasonal_at_h(self, h):
        idx = (self.t - self.m + h) % self.m
        return list(self.s)[idx]

    def tick(self, payload, performance_metrics=None):
        x = float(payload['dove_location'])
        t = payload['time']
        self.add_to_quarantine(t, x)
        prev_x = self.pop_from_quarantine(t)

        if self.l is None or self.b is None or len(self.s) < self.m:
            self.initialize(x)

        s_tm = self.s[0] if len(self.s) == self.m else 1.0
        prev_l = self.l

        # level update (multiplicative seasonal adjustment)
        self.l = self.alpha * (x / max(s_tm, 1e-6)) + (1 - self.alpha) * (self.l + self.b)
        self.l = max(self.l, 1e-6)

        # trend update
        self.b = self.beta * (self.l - prev_l) + (1 - self.beta) * self.b

        # seasonal update
        s_t = self.gamma * (x / max(self.l, 1e-6)) + (1 - self.gamma) * s_tm
        s_t = max(s_t, 1e-6)
        if len(self.s) == self.m:
            self.s.popleft()
        self.s.append(s_t)

        # residual update for scale
        if prev_x is not None:
            one_step_pred = (self.l + 1 * self.b) * self.seasonal_at_h(1)
            resid = x - one_step_pred
            self.res_ewma.update(abs(resid))

        self.t += 1

    def predict(self):
        if self.t < self.warmup or self.l is None or self.b is None or len(self.s) < self.m:
            return None
        loc = (self.l + self.b) * self.seasonal_at_h(1)
        base_scale = max(self.res_ewma.get(default=Constants.MIN_SCALE), Constants.MIN_SCALE)
        scale = min(max(base_scale, Constants.VARIANCE_FLOOR), Constants.MAX_SCALE_MULT * base_scale)
        return {
            "type":"mixture",
            "components":[
                {"density":{"type":"builtin","name":"norm","params":{"loc":float(loc),"scale":float(scale)}}, "weight":0.4},
                {"density":{"type":"builtin","name":"norm","params":{"loc":float(loc),"scale":float(scale*2)}}, "weight":0.6}
            ]
        }

# ----------------------------
# Simple dynamic ensemble wrapper (optional)
# ----------------------------
class LikelihoodWeightedEnsemble(TrackerBase):
    def __init__(self, trackers, horizon=HORIZON):
        super().__init__(horizon)
        self.trackers = trackers
        self.lik_ewmas = [EWMA(alpha=0.2) for _ in trackers]  # track recent likelihoods
        self.t = 0

    def tick(self, payload, performance_metrics=None):
        # Pass through tick to each tracker via evaluator pipeline externally
        self.t += 1

    def predict(self):
        # Combine latest predictions based on recent likelihood EWMA weights
        components = []
        weights = []
        for i, tr in enumerate(self.trackers):
            if hasattr(tr, 'latest_valid_prediction') and tr.latest_valid_prediction is not None:
                # Use evaluator to set recent likelihood EWMA outside if available
                pred = tr.latest_valid_prediction
                # Fallback to equal weight if no likelihood
                w = self.lik_ewmas[i].get(default=1.0)
                weights.append(max(w, 1e-3))
                if pred['type'] == 'mixture':
                    for c in pred['components']:
                        components.append({"density": c["density"], "weight": c["weight"] * w})
                else:
                    components.append({"density": pred, "weight": w})
        if not components:
            return None
        # Normalize weights
        total_w = sum(c["weight"] for c in components)
        for c in components:
            c["weight"] = c["weight"] / max(total_w, 1e-9)
        return {"type": "mixture", "components": components}

# ----------------------------
# Example Run: Ensemble of Holt + HW Add + HW Mul
# ----------------------------
if __name__ == "__main__":
    trackers = [
        HoltTracker(horizon=HORIZON, alpha=0.7, beta=0.25, scale_alpha=0.12),
        HoltWintersAddTracker(horizon=HORIZON, alpha=0.6, beta=0.2, gamma=0.3, m=8, scale_alpha=0.1),
        HoltWintersMulTracker(horizon=HORIZON, alpha=0.7, beta=0.25, gamma=0.3, m=12, scale_alpha=0.1),
    ]

    evaluators = [TrackerEvaluator(t) for t in trackers]
    gen = remote_test_data_generator()

    for i, payload in enumerate(tqdm(gen, desc="Running Holt & Holt-Winters Ensemble")):
        for ev in evaluators:
            ev.tick_and_predict(payload)

        # Optionally print the first tracker’s output for monitoring
        if i > 500 and evaluators[0].latest_valid_prediction is not None:
            loc, scale = get_loc_and_scale(evaluators[0].latest_valid_prediction)
            print(f"Tick {i}: loc={loc:.3f}, scale={scale:.3f}")

        if i > 2000:
            break

    for idx, ev in enumerate(evaluators):
        print(f"Tracker {idx} likelihood score: {ev.overall_likelihood_score():.4f}")


Running Holt & Holt-Winters Ensemble: 0it [00:00, ?it/s]

Tick 501: loc=2525.035, scale=0.100
Tick 502: loc=2525.036, scale=0.100
Tick 503: loc=2525.036, scale=0.100
Tick 504: loc=2525.034, scale=0.100
Tick 505: loc=2525.034, scale=0.100
Tick 506: loc=2525.034, scale=0.100
Tick 507: loc=2525.023, scale=0.100
Tick 508: loc=2525.026, scale=0.100
Tick 509: loc=2525.026, scale=0.100
Tick 510: loc=2525.034, scale=0.100
Tick 511: loc=2525.034, scale=0.100
Tick 512: loc=2525.034, scale=0.100
Tick 513: loc=2525.034, scale=0.100
Tick 514: loc=2525.034, scale=0.100
Tick 515: loc=2525.022, scale=0.100
Tick 516: loc=2525.027, scale=0.100
Tick 517: loc=2525.027, scale=0.100
Tick 518: loc=2525.027, scale=0.100
Tick 519: loc=2525.027, scale=0.100
Tick 520: loc=2525.027, scale=0.100
Tick 521: loc=2525.027, scale=0.100
Tick 522: loc=2525.042, scale=0.100
Tick 523: loc=2525.045, scale=0.100
Tick 524: loc=2525.045, scale=0.100
Tick 525: loc=2525.044, scale=0.100
Tick 526: loc=2525.046, scale=0.100
Tick 527: loc=2525.046, scale=0.100
Tick 528: loc=2525.046, scal

#Summary: This project implements finetuned Holt–Winters ensemble trackers for adaptive time series forecasting. It enhances early prediction accuracy and provides robust diagnostic insights for competitive model evaluation.